In [1]:
import numpy as np
import pandas as pd
import keras
import random
import tensorflow as tf
from sklearn import preprocessing
from sklearn import metrics
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNN, RNN, LSTM, GRU, SpatialDropout1D, Dropout

In [2]:
import matplotlib.pyplot as plt
import os, os.path, sys
import copy

In [3]:
#random.seed(7)
#np.random.seed(7)
#tf.random.set_seed(7)
keras.utils.set_random_seed(7)

In [4]:
path = r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\podaci\0k\Y"

In [5]:
dir = os.listdir(path)

In [6]:
file_names = []
for file in dir:
    file_names.append(file)

In [7]:
min_kord = [float('inf') for x in range(61000)]
max_kord = [float('-inf') for x in range(61000)]

for file in file_names:
  current_file = pd.read_csv(path+'/'+file, header=None)[:61000].astype('int')

  for i in range(len(current_file)):
    if current_file[0][i] < min_kord[i]:
      min_kord[i] = current_file[0][i]
    elif current_file[0][i] > max_kord[i]:
      max_kord[i] = current_file[0][i]

In [8]:
kord_sum = [0 for x in range(61000)]

for file in file_names:
  current_file = pd.read_csv(path+'/'+file, header=None)[:61000].astype('int')

  for i in range(len(current_file)):
    kord_sum[i] += current_file[0][i]

napravljene su prosecne koordinate

In [9]:
average_kord = [x//len(file_names) for x in kord_sum]

In [10]:
synth_path = r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\sinteticki podaci\0k\Y"

In [11]:
dir2 = os.listdir(synth_path)

In [12]:
file_names_synt = []
for file in dir2:
    file_names_synt.append(file)

In [13]:
train_synt = np.concatenate([pd.read_csv(synth_path + "/" + file, header=None)[:61000].astype('int')
                              for file in file_names_synt], axis=0)
test_synt = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in file_names], axis=0)

best step = 1

In [14]:
def create_timeseries_split(data, time_steps):

  # Get the number of samples in the data
  num_samples = len(data) - time_steps

  # Create empty arrays to store features and target values
  x_train = np.zeros((num_samples, time_steps, 1))
  y_train = np.zeros((num_samples, 1))

  # Loop through the data and create features and target values
  for i in range(num_samples):
    # Extract a slice of data for the current feature
    x_train[i] = data[i:i+time_steps, :]

    # The target value is the next value after the feature
    y_train[i] = data[i+time_steps, 0]

  return x_train, y_train

In [15]:
time_steps = 1

In [16]:
train_X, train_y = create_timeseries_split(train_synt, time_steps)

In [17]:
test_X, test_y = create_timeseries_split(test_synt, time_steps)

In [18]:
input_shape = (time_steps,1)

In [19]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)
modelMSE = Sequential()
modelMSE.add(GRU(units = 32, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 32, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 32, activation=keras.layers.LeakyReLU()))
#modelMSE.add(Dense(units = 8, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)


In [20]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 1024, verbose = 2)

Epoch 1/10

59571/59571 - 225s - loss: 56837376.0000 - 225s/epoch - 4ms/step
Epoch 2/10
59571/59571 - 223s - loss: 1472.6884 - 223s/epoch - 4ms/step
Epoch 3/10
59571/59571 - 209s - loss: 1471.7441 - 209s/epoch - 4ms/step
Epoch 4/10
59571/59571 - 199s - loss: 1471.8834 - 199s/epoch - 3ms/step
Epoch 5/10
59571/59571 - 197s - loss: 1471.0702 - 197s/epoch - 3ms/step
Epoch 6/10
59571/59571 - 223s - loss: 1470.9867 - 223s/epoch - 4ms/step
Epoch 7/10
59571/59571 - 224s - loss: 1470.9348 - 224s/epoch - 4ms/step
Epoch 8/10
59571/59571 - 224s - loss: 1469.6018 - 224s/epoch - 4ms/step
Epoch 9/10
59571/59571 - 213s - loss: 1470.6823 - 213s/epoch - 4ms/step
Epoch 10/10
59571/59571 - 190s - loss: 1469.0378 - 190s/epoch - 3ms/step


In [21]:
train_scores = modelMSE.evaluate(test_X, test_y)

57188/57188 [==============================] - 70s 1ms/step - loss: 1338.6993


In [22]:
test_predict_MSE = modelMSE.predict(test_X)

57188/57188 [==============================] - 68s 1ms/step


In [23]:
uspeli_indeksiMSE = []
num_correct_MSE = 0

for i in range(61000):
    if(min_kord[i] <= test_predict_MSE[i][0] <= max_kord[i]):
        num_correct_MSE += 1
        uspeli_indeksiMSE.append(i)

print("Procenat koliko MSE upada u pojas", num_correct_MSE/61000)
print("Koliko je MSE upalo u pojas", num_correct_MSE)

Procenat koliko MSE upada u pojas 0.9068852459016393
Koliko je MSE upalo u pojas 55320
